## 1. Setup and Installation

First, let's install and import the required packages.

In [13]:
# Install required packages if needed (uncomment if running in Colab)
# !pip install dash dash-bootstrap-components plotly pandas numpy

In [55]:
import os
import sys
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import importlib

# Set the working directory to where the modules are
# Adjust this path if needed
SCRIPT_DIR = r"c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\Agricultural_Data_Analysis"
os.chdir(SCRIPT_DIR)
sys.path.insert(0, SCRIPT_DIR)

# Import our custom modules (with reload to pick up latest changes)
import data_prep
import visuals
importlib.reload(data_prep)
importlib.reload(visuals)

from data_prep import (
    prepare_all_data, 
    load_sample_data,
    get_available_years,
    get_available_crops,
    get_available_states
)
from visuals import (
    hex_map_figure,
    state_crop_bar_chart,
    area_trend_chart,
    land_use_trend_chart,
    operations_trend_chart,
    yield_biotech_trend_chart,
    revenue_trend_chart,
    area_vs_urban_scatter,
    revenue_vs_area_bubble,
    labor_intensity_scatter,
    yield_vs_biotech_scatter,
    sector_comparison_chart,
    boom_crops_chart,
    HEX_LAYOUT
)

print("✅ Modules imported successfully!")

✅ Modules imported successfully!


## 2. Load the Data

Load and preprocess the USDA agricultural data. We'll use sample data for faster loading.

In [50]:
# Load sample data (faster for demo purposes)
# Change to prepare_all_data() for full dataset
print("Loading data... (this may take a minute)")
data = load_sample_data()

print("\n📊 Loaded datasets:")
for key, df in data.items():
    if isinstance(df, pd.DataFrame):
        print(f"  • {key}: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Extract key dataframes for use in visualizations
labor_df = data.get('labor', pd.DataFrame())
farm_ops_df = data.get('farm_operations', pd.DataFrame())

print("\n📈 Labor data available:", not labor_df.empty)
print("📈 Farm operations data available:", not farm_ops_df.empty)

Loading data... (this may take a minute)
LOADING AND PREPARING DATA
Loading: nass_crops_commodities.csv...
  Loaded 344 rows
Loading: nass_crops_field_crops.csv...
  Loaded 529,884 rows

Total combined rows: 530,228
  Loaded 529,884 rows

Total combined rows: 530,228
Aggregated state×crop×year: 14,088 rows
Loading: MajorLandUse.csv...
  Loaded 15,872 rows
Aggregated state×year land use: 816 rows
Loading: BiotechCropsAllTables2024.csv...
  Loaded 3,500 rows
Aggregated biotech state×crop×year: 900 rows
Loading LABOR data from Economics file...
Aggregated state×crop×year: 14,088 rows
Loading: MajorLandUse.csv...
  Loaded 15,872 rows
Aggregated state×year land use: 816 rows
Loading: BiotechCropsAllTables2024.csv...
  Loaded 3,500 rows
Aggregated biotech state×crop×year: 900 rows
Loading LABOR data from Economics file...
  Loaded 725 state-year labor records
Loading FARM OPERATIONS data from Economics file...
  Loaded 725 state-year labor records
Loading FARM OPERATIONS data from Economics 

In [51]:
# Quick look at the main data
state_crop_df = data.get('state_crop_year', pd.DataFrame())
if not state_crop_df.empty:
    print("State × Crop × Year Data Sample:")
    display(state_crop_df.head(10))

State × Crop × Year Data Sample:


,state_alpha,state_name,commodity_desc,year,sector_desc,ACCESSIBILITY,"ACCESSIBILITY, 5 YEAR AVG","ACCESSIBILITY, PREVIOUS YEAR",ACTIVE GINS,ACTIVITY,...,SEED FOR PLANTING,"START DATE, AVG","START DATE, MIN",STOCKS,SUCROSE,TAPS,WAREHOUSES,WATER APPLIED,yield_per_acre,"YIELD, MEDIAN"
0,AK,ALASKA,BARLEY,2001,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AK,ALASKA,BARLEY,2003,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AK,ALASKA,BARLEY,2005,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.4,NaN
3,AK,ALASKA,BARLEY,2007,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,AK,ALASKA,BARLEY,2008,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AK,ALASKA,BARLEY,2009,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AK,ALASKA,BARLEY,2010,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AK,ALASKA,BARLEY,2011,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AK,ALASKA,BARLEY,2012,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AK,ALASKA,BARLEY,2013,CROPS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.3,NaN


In [52]:
# Check available years and states
if not state_crop_df.empty:
    years = get_available_years(state_crop_df)
    states = get_available_states(state_crop_df)
    crops = get_available_crops(state_crop_df)
    
    print(f"📅 Years available: {min(years)} - {max(years)} ({len(years)} years)")
    print(f"🗺️ States available: {len(states)}")
    print(f"🌾 Crops/Commodities: {len(crops)}")
    print(f"\nTop 10 crops: {crops[:10]}")

📅 Years available: 2001 - 2025 (25 years)
🗺️ States available: 50
🌾 Crops/Commodities: 60

Top 10 crops: ['AMARANTH', 'BARLEY', 'BEANS', 'BUCKWHEAT', 'CAMELINA', 'CANOLA', 'CHICKPEAS', 'COLD STORAGE CAPACITY', 'CORN', 'COTTON']


## 3. Hex-Tile Map Visualization

The hex-tile map provides an overview of all US states, colored by agricultural metrics.

In [48]:
# Aggregate data to state level for the hex map
state_totals = state_crop_df.groupby('state_alpha').agg({
    'area_harvested_acres': 'sum'
}).reset_index()

# Create hex map
fig = hex_map_figure(
    state_totals, 
    'area_harvested_acres',
    selected_state='IA',  # Highlight Iowa
    color_scale='Tealgrn',
    title='US States by Total Area Harvested (acres)'
)
fig.show()

## 4. State-Level Analysis

Let's drill down into a specific state to see its crop distribution.

In [22]:
# Select a state to analyze
SELECTED_STATE = 'IA'  # Iowa - major agricultural state
SELECTED_YEAR = 2023   # Most recent year with data

# Get state name
state_name = HEX_LAYOUT[HEX_LAYOUT['state_alpha'] == SELECTED_STATE]['state_name'].iloc[0]
print(f"Analyzing: {state_name} ({SELECTED_STATE})")

Analyzing: Iowa (IA)


In [57]:
# Top crops by area harvested
fig = state_crop_bar_chart(
    state_crop_df, 
    SELECTED_STATE, 
    value_col='area_harvested_acres',
    year=SELECTED_YEAR,
    top_n=15
)
fig.show()

In [47]:
# Top crops by revenue (if available)
if 'revenue_usd' in state_crop_df.columns:
    fig = state_crop_bar_chart(
        state_crop_df, 
        SELECTED_STATE, 
        value_col='revenue_usd',
        year=SELECTED_YEAR,
        top_n=15,
        color_scale='Viridis'
    )
    fig.show()
else:
    print("Revenue data not available in this sample")

## 5. Trend Analysis

See how crop areas and other metrics have changed over time.

In [42]:
# Area harvested trends for top crops
fig = area_trend_chart(
    state_crop_df, 
    SELECTED_STATE,
    top_n=5
)
fig.show()

In [43]:
# Revenue trends (if available)
if 'revenue_usd' in state_crop_df.columns:
    fig = revenue_trend_chart(
        state_crop_df,
        SELECTED_STATE,
        top_n=5
    )
    fig.show()
else:
    print("Revenue data not available")

In [53]:
# Farm Operations Trend
# Uses farm_operations data from Economics dataset
fig = operations_trend_chart(
    state_crop_df,
    SELECTED_STATE,
    farm_ops_df=farm_ops_df
)
fig.show()

In [56]:
# Labor Statistics Scatter Plot
# Shows wage rate vs workers across all states
fig = labor_intensity_scatter(
    state_crop_df,
    labor_df,
    state_alpha=SELECTED_STATE,
    year=SELECTED_YEAR
)
fig.show()

## 6. Land Use Analysis

Explore how land use has changed over time, including the relationship between cropland and urban areas.

In [58]:
# Land use trends
landuse_df = data.get('landuse', pd.DataFrame())

if not landuse_df.empty:
    fig = land_use_trend_chart(landuse_df, SELECTED_STATE)
    fig.show()
else:
    print("Land use data not loaded")

In [59]:
# Cropland vs Urban change scatter
if not landuse_df.empty:
    fig = area_vs_urban_scatter(
        state_crop_df,
        landuse_df,
        state_alpha=SELECTED_STATE
    )
    fig.show()

## 7. Yield & Technology Analysis

Explore the relationship between yield improvements and biotech (GE) adoption.

In [29]:
# Biotech data
biotech_df = data.get('biotech', pd.DataFrame())

if not biotech_df.empty:
    print("Biotech adoption data sample:")
    display(biotech_df.head(10))
else:
    print("Biotech data not loaded")

Biotech adoption data sample:


trait_type,state_alpha,state_name,crop,year,pct_all_ge,pct_bt,pct_ht,pct_stacked
0,AL,Alabama,COTTON,2005,92.0,10.0,28.0,54.0
1,AL,Alabama,COTTON,2006,95.0,10.0,25.0,60.0
2,AL,Alabama,COTTON,2007,95.0,10.0,25.0,60.0
3,AL,Alabama,COTTON,2008,98.0,18.0,15.0,65.0
4,AL,Alabama,COTTON,2009,91.0,13.0,18.0,60.0
5,AL,Alabama,COTTON,2010,94.0,11.0,7.0,76.0
6,AL,Alabama,COTTON,2011,97.0,18.0,4.0,75.0
7,AL,Alabama,COTTON,2012,98.0,17.0,11.0,70.0
8,AL,Alabama,COTTON,2013,98.0,12.0,4.0,82.0
9,AL,Alabama,COTTON,2014,98.0,9.0,6.0,83.0


In [30]:
# Yield vs Biotech adoption for Corn
if not biotech_df.empty:
    fig = yield_biotech_trend_chart(
        state_crop_df,
        biotech_df,
        SELECTED_STATE,
        crop='CORN'
    )
    fig.show()

In [31]:
# Yield vs GE adoption scatter (all states)
if not biotech_df.empty:
    fig = yield_vs_biotech_scatter(
        state_crop_df,
        biotech_df,
        crop='CORN'
    )
    fig.show()

## 8. Boom Crops Analysis

Identify which crops have grown the most over the decades.

In [44]:
# Boom crops by area growth
fig = boom_crops_chart(
    state_crop_df,
    metric='area_harvested_acres',
    top_n=15
)
fig.show()

## 9. Revenue vs Area Bubble Chart

Compare revenue and area for different crops in a state.

In [35]:
fig = revenue_vs_area_bubble(
    state_crop_df,
    SELECTED_STATE,
    year=SELECTED_YEAR
)
fig.show()

## 10. Run the Full Interactive Dashboard

Launch the complete Dash application with all interactive features.

In [36]:
# Import the app module
from app import create_app, run_in_notebook

print("To run the full dashboard, execute one of the following:")
print("\n1. In this notebook (inline):")
print("   run_in_notebook(mode='inline')")
print("\n2. In a new browser tab:")
print("   run_in_notebook(mode='external')")
print("\n3. From command line:")
print("   python app.py")

To run the full dashboard, execute one of the following:

1. In this notebook (inline):
   run_in_notebook(mode='inline')

2. In a new browser tab:
   run_in_notebook(mode='external')

3. From command line:
   python app.py


In [37]:
# Uncomment the line below to run the dashboard
# Warning: This will block the notebook until you stop the server (Ctrl+C or Interrupt Kernel)

# run_in_notebook(mode='external', port=8050)

## 11. Custom Analysis

Use the data and visualization functions for your own custom analysis.

In [38]:
# Example: Compare multiple states
states_to_compare = ['IA', 'IL', 'NE', 'MN', 'IN']  # Corn Belt states

# Get total area by state
comparison = state_crop_df[
    (state_crop_df['state_alpha'].isin(states_to_compare)) &
    (state_crop_df['commodity_desc'] == 'CORN')
].groupby(['state_alpha', 'year'])['area_harvested_acres'].sum().reset_index()

# Plot
fig = px.line(
    comparison,
    x='year',
    y='area_harvested_acres',
    color='state_alpha',
    markers=True,
    title='Corn Area Harvested - Corn Belt States',
    labels={
        'year': 'Year',
        'area_harvested_acres': 'Area Harvested (acres)',
        'state_alpha': 'State'
    }
)
fig.update_layout(template='plotly_white')
fig.show()

In [39]:
# Example: National totals over time
national = state_crop_df.groupby(['year', 'commodity_desc'])['area_harvested_acres'].sum().reset_index()
top_crops = national.groupby('commodity_desc')['area_harvested_acres'].sum().nlargest(5).index
national_top = national[national['commodity_desc'].isin(top_crops)]

fig = px.area(
    national_top,
    x='year',
    y='area_harvested_acres',
    color='commodity_desc',
    title='National Area Harvested - Top 5 Crops',
    labels={
        'year': 'Year',
        'area_harvested_acres': 'Area Harvested (acres)',
        'commodity_desc': 'Crop'
    }
)
fig.update_layout(template='plotly_white')
fig.show()

---

## Summary

This dashboard provides comprehensive analysis of US agricultural data including:

1. **Geographic Overview**: Hex-tile map showing metrics by state
2. **Crop Distribution**: Bar charts of top crops by various metrics
3. **Time Series Analysis**: Trends in area, revenue, yield over decades
4. **Land Use Changes**: Relationship between cropland and urbanization
5. **Technology Impact**: Biotech adoption effects on yields
6. **Economic Analysis**: Revenue, operations, and profitability metrics

For the full interactive experience, run `python app.py` from the command line or use `run_in_notebook()` in Jupyter.